# Example: Get, inspect, change, and put back a instrument method with a binary solvent manager

Here outlines how to interact with instrument methods and module methods.

- Finding and getting a instrument method.
- Inspecting what types of module methods are in the instrument method.
- Inspecting the instrument parameters in the method, including column temperature and gradient table.
- Changing instrument parameters and posting the new method.

Get API Address

In [1]:
import os
from dotenv import load_dotenv

# get api address from .env file
load_dotenv("../.env_vars")
EMPOWER_API_ADDRESS = os.getenv("EMPOWER_API_ADDRESS_PRD")

# Finding and getting a instrument method

We first find the list of all instrument methods, select one, and get that from Empower:

In [2]:
import pprint

from OptiHPLCHandler import EmpowerHandler

# Create an instance of the EmpowerHandler class
handler = EmpowerHandler(project="WebAPI_test", address=EMPOWER_API_ADDRESS)
handler.connection.default_get_timeout = 120
handler.connection.default_post_timeout = 120

# Get the list of methods, select one, and get the method details
with handler:
    method_list = handler.GetMethodList()  # Get the list of instrument methods
    method_name = method_list[0]  # Select the first method
    print(method_name)
    full_method = handler.GetInstrumentMethod(method_name)

@BSM_PDA_Template


# Inspecting the instrument method

We can now look into the module methods in the instrument method. 

In [3]:
pp = pprint.PrettyPrinter(indent=2)

print(full_method)
print(f"Valve positions: {full_method.module_method_list[-1].valve_position}")
# Printing the valve position for the solvent manager module method in the list.
# Notice that we do not need to know the tag name to print the valve position.
print("\n\nStart of gradient table:\n")
pp.pprint(full_method.gradient_table[0:2])
# Printing the first two entries gradient table.
# Notice that we do not need to know the tag names to print the gradient table

EmpowerInstrumentMethod with 4 module methods of types PDAMethod, EmpowerModuleMethod, ColumnManagerMethod, BSMMethod
Valve positions: ['A1', 'B2']


Start of gradient table:

[ { 'CompositionA': '50.0',
    'CompositionB': '50.0',
    'Curve': 'Initial',
    'Flow': '0.300',
    'Time': 'Initial'},
  { 'CompositionA': '0.0',
    'CompositionB': '100.0',
    'Curve': '6',
    'Flow': '0.300',
    'Time': '1.00'}]


In [26]:
print("\n\nDetector settings:")
# The list of detectors found in the instrument method
print(full_method.detector_method_list)

# Check if lamp enabled
print("\nLamp status:")
print(full_method.detector_method_list[0].lamp_enabled)

# Check the channel settings
print("\nChannel settings:")
pprint.pprint(full_method.detector_method_list[0].channel_dict)



Detector settings:

Lamp status:
True

Channel settings:
{'Channel1': {'Enable': True,
              'Type': 'Single',
              'Wavelength1': '214',
              'XML': '<Channel1>\r\n'
                     '    <Enable>true</Enable>\r\n'
                     '    <DataMode>DataModeAbsorbance_0</DataMode>\r\n'
                     '    <Wavelength1>214</Wavelength1>\r\n'
                     '    <Wavelength2>498</Wavelength2>\r\n'
                     '    <Resolution>Resolution_48</Resolution>\r\n'
                     '    <Ratio2DMinimumAU>0.01</Ratio2DMinimumAU>\r\n'
                     '  </Channel1>'},
 'Channel2': {'Enable': False,
              'Type': 'Single',
              'Wavelength1': '254',
              'XML': '<Channel2>\r\n'
                     '    <Enable>false</Enable>\r\n'
                     '    <DataMode>DataModeAbsorbance_0</DataMode>\r\n'
                     '    <Wavelength1>254</Wavelength1>\r\n'
                     '    <Wavelength2>254</Wav

Note that most of the methods are simply EmpowerModuleMethods. That is the generic
type for module methods that aren't specifically accounted for. At the moment, that
is every type of method expect:
 - Solvent manager methods (BSMMethod and QSMMethod), which have the special properties `valve_position` and `gradient_table`.
 - Column oven methods (SampleManagerMethod), which have the special property `column_temperature`.

All of the special properties can be accessed as set from the `EmpowerInstrumentMethod`.
Getting `EmpowerInstrumentMethod.column_temperature` will produce an error if the
instrument method controls several column ovens (e.g. one in a sample manager and one in
a column manager), and the column ovens have different temperatures. Setting 
`EmpowerInstrumentMethod.column_temperature` will set the temperature for all column
ovens if there are any, and produce an error otherwise.

Here is an example for a quaternary system:

In [9]:
import pprint

from OptiHPLCHandler import EmpowerHandler

# Create an instance of the EmpowerHandler class
handler = EmpowerHandler(project="WebAPI_test", address=EMPOWER_API_ADDRESS)

# Get the list of methods, select one, and get the method details
with handler:
    method_list = handler.GetMethodList()  # Get the list of instrument methods
    method_name = method_list[2]  # Select the first method
    print(method_name)
    full_method = handler.GetInstrumentMethod(method_name)

@QSM_PDA_Template


In [14]:
pp = pprint.PrettyPrinter(indent=2)

print(full_method)
print(f"Valve positions: {full_method.module_method_list[0].valve_position}")
# Printing the valve position for the solvent manager module method in the list.
print("\n\nStart of gradient table:\n")
pp.pprint(full_method.gradient_table[0:2])
# Printing the first two entries gradient table.

EmpowerInstrumentMethod with 4 module methods of types QSMMethod, EmpowerModuleMethod, EmpowerModuleMethod, ColumnManagerMethod
Valve positions: ['A0', 'B0', 'C0', 'D0']


Start of gradient table:

[ { 'CompositionA': '90.0',
    'CompositionB': '10.0',
    'CompositionC': '0.0',
    'CompositionD': '0.0',
    'Curve': 'Initial',
    'Flow': '0.400',
    'Time': 'Initial'},
  { 'CompositionA': '90.0',
    'CompositionB': '10.0',
    'CompositionC': '0.0',
    'CompositionD': '0.0',
    'Curve': '6',
    'Flow': '0.400',
    'Time': '1.00'}]


# Changing instrument parameters and posting the new method.

We can also change the values of the instrument method, and post the changed method back
to Empower, so we can use it to analyse samples.

Remember to give the method a new name before posting it to Empower. OptiHPLCHandler
does not allow changing methods in Empower, only creating new ones.

In [ ]:
gradient_table = full_method.gradient_table  # Get the gradient table
for step in gradient_table:
    step["Flow"] = 0.5  # Set the flow to 0.5 mL/min for all steps
gradient_table[1]["Time"] = 5 / 3  # Set the time for the second step to 5/3 minutes.
# Notice the warning that 1.666666667 minutes is rounded to 1.667, since Empower will
# misinterpret values with too many decimals.
full_method.gradient_table = (
    gradient_table  # Set the gradient table to the updated gradient table
)
full_method.valve_position = [
    "A2",
    "B1",
]  # Set the valve position to A2 and B1. You can also set only one of the valves.
full_method.method_name = "New method name"  # Set the method name
with handler:
    handler.PostInstrumentMethod(full_method)  # Post the updated method to Empower

# MethodSet methods

In order to run an instrument method, we need a methodset method that includes it.

Let's start by logging in, getting the list of all methodset methods present, and look
at one of them

In [3]:
with handler:
    methodset_method_list = handler.GetMethodList("MethodSet")
    print(methodset_method_list)
    methodsset_method = handler.GetMethodSetMethod(methodset_method_list[0])
methodsset_method

['@BSM_PDA_Template', '@BSM_TUV_Template', '@QSM_PDA_Template', 'B104_11_IClass_280nm_UPLC12', 'BSM_Sys12_Original', 'CMS_test_23460013_1', 'CMS_test_23460013_2']


{'name': '@BSM_PDA_Template',
 'isLocked': False,
 'id': 1463,
 'version': 1,
 'date': '2023-11-10T08:18:55',
 'comments': '',
 'modifiedBy': 'SLCB',
 'revisionHistory': [{'id': 1463,
   'version': 1,
   'modificationDate': '10-Nov-2023 08:18:55 CET',
   'user': 'SLCB',
   'comment': "Created method '@BSM_PDA_Template'."}],
 'instrumentMethod': '@BSM_PDA_Template'}

We can now create a simple MethodSet method

In [7]:
method_set_method = {
    "name": "New methodset method",
    "instrumentMethod": full_method.method_name,
}
with handler:
    handler.PostMethodSetMethod(method_set_method)

We can also specify more about the methodset method - Export methods, and default processing and reporting methods

In [6]:
with handler:
    processing_method_list = handler.GetMethodList("Processing")
    report_method_list = handler.GetMethodList("Report")
    method_set_method = {
        "name": "Complex new methodset method",
        "instrumentMethod": full_method.method_name,
        "defaultProcessingMethod": processing_method_list[0],
        "defaultReportingMethod": report_method_list[0],
    }
    handler.PostMethodSetMethod(method_set_method)